In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
from collections import Counter

In [2]:
df = pd.read_csv("data_RAW_recipes.csv")

In [3]:
# name, steps, ingredients

df.head(3)


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            231636 non-null  object
 1   id              231637 non-null  int64 
 2   minutes         231637 non-null  int64 
 3   contributor_id  231637 non-null  int64 
 4   submitted       231637 non-null  object
 5   tags            231637 non-null  object
 6   nutrition       231637 non-null  object
 7   n_steps         231637 non-null  int64 
 8   steps           231637 non-null  object
 9   description     226658 non-null  object
 10  ingredients     231637 non-null  object
 11  n_ingredients   231637 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 21.2+ MB


In [5]:
df = df.dropna()

In [6]:
len(df)

226657

In [7]:
def validate_symbols(not_allowed):
    def fun(lestring):
        fnd = [e for e in not_allowed if e in lestring]
        return len(fnd) == 0

    return fun


In [8]:
not_allowed_row = [
 '.', #dev filtrować na poziomie df TODO
 ':', #dev filtrować na poziomie df TODO
 ';', #dev filtrować na poziomie df TODO
]

In [9]:
df = df[np.vectorize(validate_symbols(not_allowed_row))(df.steps)]

In [10]:
len(df)

202475

In [11]:
print(df[df.n_steps<=20][df.n_steps>=1].groupby(df.n_steps)["id"].count())
df = df[df.n_steps<=12][df.n_steps>=5] #dev
len(df)

n_steps
1      2362
2      6749
3     10951
4     13984
5     17058
6     18644
7     19254
8     18395
9     16561
10    14496
11    12378
12    10233
13     8453
14     6730
15     5373
16     4228
17     3447
18     2632
19     2228
20     1695
Name: id, dtype: int64


/tmp/ipykernel_26705/1192130333.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[df.n_steps<=20][df.n_steps>=1].groupby(df.n_steps)["id"].count())
/tmp/ipykernel_26705/1192130333.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[df.n_steps<=12][df.n_steps>=5] #dev


127019

In [12]:
print(df[df.n_ingredients<=20][df.n_ingredients>=1].groupby(df.n_ingredients)["id"].count())
df = df[df.n_ingredients<=10][df.n_ingredients>=4]
len(df)

n_ingredients
1        10
2       909
3      3073
4      6514
5     10595
6     13332
7     15291
8     15749
9     14903
10    12705
11    10465
12     7791
13     5507
14     3694
15     2454
16     1590
17      939
18      605
19      353
20      202
Name: id, dtype: int64


/tmp/ipykernel_26705/2601168383.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[df.n_ingredients<=20][df.n_ingredients>=1].groupby(df.n_ingredients)["id"].count())
/tmp/ipykernel_26705/2601168383.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[df.n_ingredients<=10][df.n_ingredients>=4]


89089

Pomysły na prunning:
- zrobić listę zezwolonych symboli
- jeżeli na końcu jest inny znak interpunkcyjny to nie dodawać "."

In [13]:
df.steps = df.steps.apply(lambda x: literal_eval(x))

In [14]:
def concatenate_steps(steps):
    steps = [list(s) for s in steps]
    steps = [[step[0].upper()] + step[1:] + ["."] for step in steps] # dev jeżeli na końcu jest inny znak interpunkcyjny to nie dodawać "."
    steps = ["".join(step) for step in steps]
    return " ".join(steps)

def concatenate_listed(listed):
    return ", ".join(literal_eval(listed))

def recipe_representation(recipe):
    rep = f"Name: {recipe['name']} ; Ingredients: {concatenate_listed(recipe['ingredients'])} ; Preparation: {concatenate_steps(recipe['steps'])}"
    f_symbols = [",", "?", "!"]
    for e in f_symbols:
        rep = rep.replace(f" {e}", f"{e}")
    rep = rep.replace(f"!.", f"!")
    rep = rep.replace(f"?.", f"?")
    rep = " ".join(rep.split())
    return rep


In [15]:
recipes_rep = df.apply(recipe_representation, axis=1)

In [16]:
c = Counter("".join(recipes_rep))
c

Counter({' ': 8670461,
         'e': 4944128,
         'a': 3330262,
         't': 3106799,
         'o': 2967996,
         'n': 2935740,
         'r': 2867897,
         'i': 2829075,
         's': 2336362,
         'l': 1887539,
         'd': 1567832,
         'h': 1407053,
         'c': 1383169,
         'u': 1263593,
         'p': 1149322,
         'm': 1066817,
         'g': 999873,
         ',': 927784,
         'b': 711903,
         '.': 697920,
         'f': 618620,
         'w': 547317,
         'k': 526204,
         'v': 392386,
         'y': 370740,
         ':': 267267,
         ';': 178178,
         'P': 177496,
         'x': 144296,
         'I': 132079,
         'S': 115296,
         '1': 104530,
         'N': 90549,
         '-': 90460,
         '0': 88916,
         '5': 80971,
         'C': 79249,
         'A': 75708,
         '2': 74504,
         '3': 71427,
         'j': 65013,
         'z': 63197,
         'B': 57892,
         '4': 41556,
         'R': 38746,
       

In [17]:
not_allowed = [
#  ' ',
#  'e',
#  'a',
#  't',
#  'n',
#  'o',
#  'r',
#  'i',
#  's',
#  'l',
#  'd',
#  'c',
#  'h',
#  'u',
#  'p',
#  'm',
#  'g',
#  ',',
#  'b',
#  '.', #dev filtrować na poziomie df TODO
#  'f',
#  'w',
#  'k',
#  'v',
#  'y',
#  ':', #dev filtrować na poziomie df TODO
#  ';', #dev filtrować na poziomie df TODO
#  'P',
#  'x',
#  'I',
#  'S',
#  '1',
#  'N',
#  '-', #dev większość przepisów go ma
#  '0',
#  '5',
#  'C',
#  'A',
#  '2',
#  '3',
#  'j',
#  'z',
#  'B',
#  '4',
#  'M',
#  'T',
#  'R',
#  'q',
 '/', #dev very popular # can be deleted -12k
#  'D',
#  'W',
#  'L',
#  'F',
#  '8',
#  'H',
#  '&',
#  'G',
#  '9',
#  '7',
 "'", #dev 's w j angielskim
#  '6',
 '!', #dev
 '"',
#  'O',
#  'U',
#  'E',
#  'Y',
#  'K',
#  'J',
 '%',
 ')',
#  'V',
 '@',
#  'Q',
 '(',
 '~',
 '#',
 '[',
 '=',
 ']',
 '+',
 '?',
#  'X',
#  'Z',
 '`',
 '}',
 '{',
 '>',
 '^',
 '<',
 '_',
 '\\',
 '$',
 '|',
 '*']

In [18]:
filtered = recipes_rep[np.vectorize(validate_symbols(not_allowed))(recipes_rep)]
len(filtered), len(recipes_rep)

(52326, 89089)

In [19]:
fin = filtered[np.vectorize(lambda x: len(x.split(" ")) <= 250)(filtered.values)]
len(fin)

52319

In [20]:
fin.to_csv("food_com_p_v1.csv")